### Reading Data From Kaggle

In [1]:
cd /Users/patriciaporal/Downloads/archive

/Users/patriciaporal/Downloads/archive


In [2]:
# Import libraries
import pandas as pd
import sklearn
from sklearn.decomposition import TruncatedSVD
import numpy as np
import dill

In [3]:
# Read data set
orders = pd.read_csv('orders.csv')
order_items = pd.read_csv('orders_items.csv')
products = pd.read_csv('products.csv')

In [4]:
# Data Cleaning
file = orders[['id','customer_id','total_line_items_price']]
items = order_items[['order_id','product_id','price','quantity']]
items.rename(columns = {'order_id':'id'}, inplace = True)
items['product_id'] = items['product_id'].astype(str)
items['product_id'] = items['product_id'].str.split('.').str[0]
cols = ['id']
file = file.join(items.set_index(cols), on=cols)
products = products.drop(['title','created_at','published_at'], axis = 1)
products.rename(columns = {'id':'product_id'}, inplace = True)
products['product_id'] = products['product_id'].astype(str)
cols = ['product_id']
file = file.join(products.set_index(cols), on=cols)
file['product_type'] = file['product_type'].str.lower()
file.loc[(file.product_type == 'hooide'),'product_type']='hoodie'
file.loc[(file.product_type == 'tousers'),'product_type']='trousers'

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-4-209952b2833a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items['product_id'] = items['product_id'].astype(str)
<ipython-input-4-209952b2833a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [5]:
summary = file.groupby('product_id').agg(orders=('id', 'nunique'), quantity=('quantity', 'sum')).sort_values(by='orders', ascending=False)

In [6]:
summary

,orders,quantity
product_id,,
12927690655,1539,1592
12927632799,1099,1134
12927629215,1078,1125
12927625695,906,968
375446541821,757,900
...,...,...
1183022388733,6,6
1183021864445,5,6
1183021929981,4,4


In [7]:
item = file.pivot_table(index='customer_id', columns=['product_id'], values='quantity').fillna(0)

In [8]:
transposed_item = item.T

In [9]:
transposed = transposed_item

In [10]:
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(transposed_item)

In [11]:
correlation_matrix = np.corrcoef(decomposed_matrix)

In [12]:
def recommend_item(i):
    item=[]
    product_id = list(transposed_item.index)
    product_ID = product_id.index(i)
    correlation_product_ID = correlation_matrix[product_ID]
    Recommend = list(transposed_item.index[correlation_product_ID > 0.90])
    Recommend.remove(i)
    a = Recommend[0:3]
    #b = products.loc[products['product_id'] == a[0]]
    for i in range(len(a)):
        item.append(products.loc[products['product_id'] == a[i]])
    print(pd.concat(item))

In [13]:
filename = 'initial_recommender.pickle'
dill.dump(recommend_item, open(filename, 'wb'))

In [14]:
recommend_item('1007443645949')

       product_id product_type
80  1007443056125       Blouse
83  1007443187197         mini
86  1007443219965       Jacket
